In [15]:
# imports:
from  random import randint ,choice ,random

In [16]:
# global variables:
unary_operators_list =list(["sin" , "cos" , "tan" , "cot" , "exp" ])
binary_operators_list =list(["+" , "-" , "/" , "*" , "**"])

# user's values:
operators_list =["+" , "-" ]*5+\
["/" , "*" ]* 3 +\
[ "**"  ]*1 +\
[ "sin" ,"cos" , "tan" , "cot" , "exp"]*0   #user can change the quantity of operators based on function determination of complexity.
users_variables = ["x"]             #what variables our function does have
min_operatos_num = 1                #determine how many operators do we need in our function(create n operators between these 2 variables.)
max_operatos_num =4
variable_or_constant_factor = 0.3   #a factor that determines to use constant or a variale in leaf node of the tree .the closer value to 0 the more constants will appear in your function.(must be between 0 and1)
depth_or_breadth_factor = 0.5       #a factor that determines to fill both sides of the operator on not. the more closer to 0 tree will get filled in breadth not depth. (must be between 0 and1)
                                    # for example: closer to 0: (((x*10)+2)-5)/3      closer to 1: ((x*10)+2) - ((x-5)\3)
min_constant_limit = 0              #determine that constants must be in what period.
max_constant_limit =10            
num_of_population_in_a_gen = 50     #how many trees should be included in each generation.

# function values:
sample_values = {                   #user must enter some values in spots to 
    "x" : [0,1,2,3,4] ,
    "outputs" : [0,1,4,9,16]        #can be skipped with replacing function in function below.
}

In [40]:
# funtion to  fill sample["output"] for 1 variable:
def fill_sample_output(func_str):
    temp_list = []
    for x_val in sample_values["x"]:
        temp_str = func_str.replace("x" , str(x_val))
        temp_val =eval(temp_str)
        temp_list.append(temp_val)
    # sample_values["outputs"]  =temp_list
    return temp_list

In [42]:
x = fill_sample_output("x**2+1")
x


[1, 2, 5, 10, 17]

In [19]:
class Node():
    def __init__(self , value=None , used=0, father=None , right=None ,left=None , depth = 0 ):
        self.father =father
        self.right = right
        self.left = left
        self.depth =depth
        self.used = used
        self._value = value
        # self.is_operator = new_value in operators_list
        self.is_op()
        self.is_binary_op()
    
    def set_random_op(self):
        new_value= choice(operators_list)                
        self.value=new_value 
    def is_binary_op(self):
        self.is_binary = self.value in binary_operators_list
    def is_op(self  ):
        self.is_operator = self.value in operators_list
    
    @property
    def value(self):
        return self._value 
    @value.setter
    def value(self, new_value=None):
        self._value = new_value
        self.is_operator = new_value in operators_list
        self.is_binary = new_value in binary_operators_list
        # self.is_binary()
        # self.is_op()
        
    def __repr__(self) -> str:
        return f"(V:{self.value} L:{self.left} R:{self.right})"
    def __str__(self) -> str:
        if self.left == None and self.right==None :                                          #constant or variable
            return str(self.value)
        elif self.value in binary_operators_list:                                            #binary operator
            return  f"({self.left} {self.value} {self.right})"
        else:                                                                                #unary operator                                        
            return f"{self.value} ({self.left}) "

In [20]:
class Tree():
    def __init__(self ,  root:Node , nodes:set , MSE = 100000 ):
        self.root = root 
        self.MSE = MSE
        self.nodes = set(nodes)
        self.nodes.add(root)
    def add_left_child(self , father_node:Node , left_val:str):
        left_node  = Node(value=left_val ,father = father_node)
        father_node.left = left_node
        left_node.depth = father_node.depth +1
        self.nodes.add(left_node)
    def add_right_child(self , father_node:Node , right_val:str):
        right_node = Node(value=right_val, father = father_node)
        father_node.right = right_node
        right_node.depth  = father_node.depth +1
        self.nodes.add(right_node)
    def __repr__(self) -> str:
        return f"tree[ {self.root} ] "

In [21]:
def create_random_tree():
    num_of_ops = randint(min_operatos_num , max_operatos_num)
    unattached_nodes =list([Node() for _ in range(num_of_ops-1)])
    root_node = Node(father="is_root",used=0 ,depth=0,value=choice(operators_list))          
    attached_nodes= list([root_node])                                                                       #a node gets attached when it has a parent
    # creating a random tree with operators:
    # print("test r:",root_node ,"n.o.o:",num_of_ops , "a:", attached_nodes , "u:" , unattached_nodes )     #>>>for logging
    while unattached_nodes !=[]:
        # print("***first loop")                                                                            #>>>for logging
        node =choice(attached_nodes)
        if node.used == 0:                                                                                  #node is binary operator.
            if node.value == None:                                                                          #sometimes it is already set in previous loop.
                node.value = choice(operators_list)            
            if node.value in binary_operators_list:
                if random()< depth_or_breadth_factor and len(unattached_nodes)>1:                           #fill both childs
                    child1 = unattached_nodes.pop()
                    child2 = unattached_nodes.pop()
                    child1.set_random_op()
                    child2.set_random_op()
                    node.left =child1
                    node.right =child2
                    child2.depth , child1.depth =node.depth +1,node.depth +1
                    node.used = 1
                    attached_nodes.append(child1)
                    attached_nodes.append(child2)
                else:                                                                                       #fill one child
                    child = unattached_nodes.pop()
                    child.set_random_op()
                    if random()<0.5:
                        node.left = child
                    else:
                        node.right = child
                    child.depth =node.depth +1
                    node.used =1
                    attached_nodes.append(child)
            else:                                                                                          #node is unary operator.
                child = unattached_nodes.pop()
                child.set_random_op()
                node.left =child
                child.depth =node.depth +1
                node.used=1
                attached_nodes.append(child)
    # print("after op nodes set: a:" ,attached_nodes ,"u:",unattached_nodes )                              #>>>for logging
    new_attached_nodes = []
    # adding values and constants to leaf nodes
    for node in attached_nodes:
        # print("in second loop; a:" ,attached_nodes ,"u:",unattached_nodes )                              #>>>for logging
        if node.right ==None :                                                                             #right node is empty
            choose_const_or_var = random()
            if  choose_const_or_var>variable_or_constant_factor:                                           #fill with constant
                temp_value = randint(min_constant_limit , max_constant_limit)
                temp_node = Node(value=temp_value ,father=node , depth=node.depth+1)
                node.right = temp_node
                new_attached_nodes.append(temp_node)
            else:                                                                                          #fill with variable                                  
                temp_value = choice(users_variables)
                temp_node = Node(value=temp_value ,father=node , depth=node.depth+1)
                node.right = temp_node
                new_attached_nodes.append(temp_node)
        if node.left ==None :                                                                              #left node is empty
            choose_const_or_var = random()
            if choose_const_or_var>variable_or_constant_factor:                                            #fill with constant
                temp_value = randint(min_constant_limit , max_constant_limit)
                temp_node = Node(value=temp_value ,father=node , depth=node.depth+1)
                node.left = temp_node
                new_attached_nodes.append(temp_node)
            else:                                                                                          #fill with variable                                  
                temp_value = choice(users_variables)
                temp_node = Node(value=temp_value ,father=node , depth=node.depth+1)
                node.left = temp_node
                new_attached_nodes.append(temp_node)
    nodes  = set(attached_nodes + new_attached_nodes)
    # print("len:",len(nodes),"nodes:",nodes)   \n",                                                       #>>>for logging
    temp_tree = Tree(root= root_node ,nodes = nodes)
    # print( "at last a:" ,attached_nodes ,"u:",unattached_nodes )                                         #>>>for logging
    
    return temp_tree

In [22]:
temp_tree = create_random_tree()
# temp_tree 

In [23]:
temp_tree

tree[ (7 - x) ] 

In [24]:
trees_list_this_gen = []
# create the genaration from create_random_tree()
for i in range(num_of_population_in_a_gen):
    temp_tree = create_random_tree()
    trees_list_this_gen.append(temp_tree)

In [25]:
trees_list_this_gen

[tree[ (3 + 2) ] ,
 tree[ (6 + ((4 - 2) - (8 - x))) ] ,
 tree[ ((x - x) - (x / x)) ] ,
 tree[ ((7 * 4) + (4 / 10)) ] ,
 tree[ (4 / 5) ] ,
 tree[ (1 + x) ] ,
 tree[ (7 * x) ] ,
 tree[ (7 * (9 / (9 + x))) ] ,
 tree[ (((3 * x) + 4) / (0 + 1)) ] ,
 tree[ (9 * 1) ] ,
 tree[ (((8 * 0) / 4) + (8 + 2)) ] ,
 tree[ ((1 - 9) ** 6) ] ,
 tree[ (9 + (x + (3 / (1 - 9)))) ] ,
 tree[ ((7 - 5) * 2) ] ,
 tree[ ((2 / 4) ** (1 + 9)) ] ,
 tree[ (x - x) ] ,
 tree[ (5 / 9) ] ,
 tree[ ((1 - 1) / (0 - x)) ] ,
 tree[ (x / (x + 4)) ] ,
 tree[ ((x + 3) + 8) ] ,
 tree[ ((4 * 4) / (3 - (10 / 8))) ] ,
 tree[ ((9 * (x + 0)) + (x - x)) ] ,
 tree[ (1 - 8) ] ,
 tree[ (x + x) ] ,
 tree[ (9 + (x / (7 * 8))) ] ,
 tree[ ((6 * x) - 0) ] ,
 tree[ (((10 + 4) - x) + 8) ] ,
 tree[ (10 - 10) ] ,
 tree[ (x + (3 * 1)) ] ,
 tree[ (x - 9) ] ,
 tree[ (8 / (5 - 8)) ] ,
 tree[ ((x + 5) - (6 * 4)) ] ,
 tree[ (((1 / x) / (4 * 7)) + x) ] ,
 tree[ ((8 - 6) + (9 * 1)) ] ,
 tree[ (2 * x) ] ,
 tree[ ((7 + 8) / (9 + 9)) ] ,
 tree[ (((8 - (x - 1)

In [26]:
trees_list_this_gen.clear()


# Failed functions and useless codes
----------------------------

##### Instance: (3 * x) + (x-4)
#### create tree from : + * - 3 x x 4




In [25]:
def calculate(node:Node):
    # node = tree.root
    if node.is_operator:
        return f"( ({calculate(node.left)} ) {node.value} ( {calculate(node.right)}) )"
    return node.value

In [ ]:
# def create_tree(tree:Tree , parse_input:str ):
#     inputs = parse_input.split(" ")
#     # twos_submultiples = [2**i for i in range(5) ]
#     index = 0
#     tree_spaces = []
#     # make space for each level and fill them with inputs
#     for i in range(5):
#         tmp = []
#         for j in range(2**i) :
#             if index<len(inputs):
#                 tmp.append(inputs[index])
#                 index +=1
#         tree_spaces.append(tmp)
#     index = 0
#     for i in range(5):
#         tmp_index = 0
#         tmp_child_detector = "l"
#         for j in range(i**2):
#             if index<len(inputs):
#                 if tree_spaces[i][j] in operators_set:
#                     if tmp_child_detector =="l":    
#                         tmp_child_detector = "r"
#                     elif tmp_child_detector=="r":
#                         tmp_child_detector = "l"

In [ ]:
# root_node = Node(value="+" )
# first_tree = Tree(root_node)
# create_tree(first_tree, "+ * - 3 x x 4")

In [ ]:
# first_tree.add_left_child("+")
# first_tree.add_left_child("-")

In [ ]:
"""failed create_tree function """
# def create_tree(tree:Tree , parse_input:str , root_node:Node):
#     current_op = None
#     reserved_op = None
#     node = root_node
#     inputs = list(parse_input.split(" "))
#     # current_op =inputs[0]  if inputs[0] in operators_set else None
#     # reserved_op = inputs[1] if inputs[1] in operators_set else None
    
#     num_index = 0
#     while True:
#         if inputs[num_index] not in operators_set :
#             num_index +=1
#         else :
#             break
#     op_index =0
    
#     while(True):
#         if (reserved_op in operators_set):
#             tree.add_left_child(node , inputs[num_index])
#             tree.add_left_child(node , inputs[num_index+1])
#             num_index = num_index+2
#             if inputs[op_index] in operators_set: 
#                 node = 
#                 op_index +=1
#             else:
#                 break


In [ ]:
"""tree_spaces: [[1] [2] [4] [8]]"""

# tree_spaces = []
# for i in range(5):
#     tmp = [None for i in range(2**i)]
#     tree_spaces.append(tmp)
        
# tree_spaces 